# Tratando dados Faltantes
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](https://drive.google.com/file/d/1-Y4bPzqbLlpYlICatgsgen99D2F1JYTz/view?usp=drive_link). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS) ou [aqui](https://drive.google.com/file/d/1-XPr96H_ik4zGQzugM8iH_onWHXMMOoZ/view?usp=drive_link).

### Objetivo
Deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.



#### Preparação da base
1. Carregar a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

In [ ]:
import pandas as pd
import requests

# carregando a base de dados
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

2. Contando o número de valores *missing* por variável.

In [ ]:
#2)
sinasc.isna()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
27024,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
27025,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
27026,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


3. Criando uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refazendo a contagem de valores *missings*.  

In [ ]:
# 3)
base_select = sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
base_select


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [ ]:
#Contando valores missing

base_select.isna()
#nenhum dado null

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
27023,False,False,False,False,False,False,False,False,False
27024,False,False,False,False,False,False,False,False,False
27025,False,False,False,False,False,False,False,False,False
27026,False,False,False,False,False,False,False,False,False


4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Removendo todos os registros com Apgar5 não preenchido. Para esta seleção, e contando novamente o número de linhas e o número de *missings*.  

In [ ]:
# 4)
#base_select['APGAR5']
base_select.dropna(subset=['APGAR5'],inplace=True)
#sinasc.dropna(subset=['APGAR5'],inplace=True)
#base_select.dropna(base_select['APGAR5'])
#base_select

In [ ]:

base_select['APGAR5'].isna()
#base_select


0        False
1        False
2        False
3        False
4        False
         ...  
27023    False
27024    False
27025    False
27026    False
27027    False
Name: APGAR5, Length: 26925, dtype: bool

5. observando as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>

In [ ]:
# 5)

base_select['ESTCIVMAE'].fillna(9.0)

0        5.0
1        2.0
2        9.0
3        5.0
4        2.0
        ... 
27023    2.0
27024    1.0
27025    2.0
27026    2.0
27027    5.0
Name: ESTCIVMAE, Length: 26925, dtype: float64

In [ ]:
base_select['CONSULTAS'].fillna(9)

2        4
3637     4
3713     3
3735     4
4108     4
        ..
26542    4
26548    4
26759    3
26791    4
26973    3
Name: CONSULTAS, Length: 144, dtype: int64

In [ ]:
base_select['ESTCIVMAE'].isna()



2        False
3637     False
3713     False
3735     False
4108     False
         ...  
26542    False
26548    False
26759    False
26791    False
26973    False
Name: ESTCIVMAE, Length: 144, dtype: bool

In [ ]:
base_select['CONSULTAS'].isna()

2        False
3637     False
3713     False
3735     False
4108     False
         ...  
26542    False
26548    False
26759    False
26791    False
26973    False
Name: CONSULTAS, Length: 144, dtype: bool

6. Substituir os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.

In [ ]:
# 6)
base_select['QTDFILVIVO'].fillna(0)

2        2.0
3637     0.0
3713     0.0
3735     2.0
4108     0.0
        ... 
26542    1.0
26548    2.0
26759    0.0
26791    1.0
26973    0.0
Name: QTDFILVIVO, Length: 144, dtype: float64

In [ ]:
base_select['QTDFILVIVO'].isna()

2        False
3637     False
3713     False
3735     False
4108     False
         ...  
26542    False
26548    False
26759    False
26791    False
26973    False
Name: QTDFILVIVO, Length: 144, dtype: bool

7. Das restantes, decidir que valore te parecem mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  

In [ ]:
# 7)
#nesse caso o melhor a fazer é deixar como valor faltante, pois os dados são sensíveis e trás um contexto sobre a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal.

8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>

In [ ]:
# 8)
#O Apgar possui uma classificação indicando se o bebê passou por asfixia:

#Entre 8 e 10 está em uma faixa 'normal'.
#Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
#Entre 4 e 5 significa 'asfixia moderada'.
#Entre 0 e 3 significa 'asfixia severa'.

base_select.loc[(base_select['APGAR5'] > 0) & (base_select['APGAR5'] <= 3), 'FAIXA_ASFIXIA'] = 'asfixia severa'
base_select.loc[(base_select['APGAR5'] >= 4) & (base_select['APGAR5']<=5), 'FAIXA_ASFIXIA'] = 'asfixia moderada'
base_select.loc[(base_select['APGAR5'] >= 6 ) & (base_select['APGAR5']<=7), 'FAIXA_ASFIXIA'] = 'asfixia leve'
base_select.loc[(base_select['APGAR5'] >= 8) & (base_select['APGAR5'] <=10), 'FAIXA_ASFIXIA'] = 'normal'
base_select[base_select['APGAR5']==10]


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,FAIXA_ASFIXIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27007,1,24,1.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
27008,1,30,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
27009,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
27015,1,25,2.0,8 a 11 anos,1.0,37 a 41 semanas,Dupla,4,10.0,normal


In [ ]:
#frequencia
count = base_select.groupby(['FAIXA_ASFIXIA']).count()
count

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,FAiXA_ASFIXIA
FAIXA_ASFIXIA,,,,,,,,,,
asfixia leve,320,320,314,317,309,306,319,320,320,320
asfixia moderada,68,68,68,65,63,61,68,68,68,0
asfixia severa,57,57,56,57,54,52,57,57,57,0
normal,26463,26463,26155,26159,24917,25274,26388,26463,26463,0


9. Renomeando as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, usando a opção ```inplace = True```.

In [ ]:
# 9) seu código aqui
#Renomeando as variáveis para que fiquem no snake case, ou seja,
#em letras minúsculas, com um underscore entre as palávras.


base_select.rename(columns = {'LOCNASC':'loc_nasc',
                              'IDADEMAE':'idade_mae',
                              'ESTCIVMAE':'est_cvi_mae',
                              'ESCMAE':'est_mae',
                              'QTDFILVIVO':'qtd_fil_vivo',
                             'GESTACAO':'gestacao',
                             'GRAVIDEZ':'gravidez',
                             'CONSULTAS':'consultas',
                             'APGAR5':'apgar_5',
                             'FAIXA_ASFIXIA':'faixa_asfixia'}, inplace=True)

base_select


,loc_nasc,idade_mae,est_cvi_mae,est_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,faixa_asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
